In [6]:
import pandas as pd
import sqlite3


In [13]:
print("\n[ S1 ] Reading Orders File\n")

orders_data = pd.read_csv("orders.csv")
print(orders_data.head())

print("\nOrders Columns:")
print(list(orders_data.columns))

print("\nOrders Dataset Summary:")
orders_data.info()



[ S1 ] Reading Orders File

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  

Orders Columns:
['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']

Orders Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-n

In [14]:
print("\n[ S2 ] Reading Users File\n")

users_data = pd.read_json("users.json")
print(users_data.head())

print("\nUsers Columns:")
print(list(users_data.columns))

print("\nUsers Dataset Summary:")
users_data.info()



[ S2 ] Reading Users File

   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold

Users Columns:
['user_id', 'name', 'city', 'membership']

Users Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


In [15]:
print("\n[ S3 ] Loading Restaurant Data\n")

db_conn = sqlite3.connect("restaurants.db")
db_cursor = db_conn.cursor()

# remove old table if exists
db_cursor.execute("DROP TABLE IF EXISTS restaurants")

with open("restaurants.sql", "r") as sql_file:
    sql_script = sql_file.read()
    db_cursor.executescript(sql_script)

db_conn.commit()

restaurant_data = pd.read_sql(
    "SELECT * FROM restaurants",
    db_conn
)

print(restaurant_data.head())
print("\nRestaurant Dataset Summary:")
restaurant_data.info()


[ S3 ] Loading Restaurant Data

   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8

Restaurant Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_id    500 non-null    int64  
 1   restaurant_name  500 non-null    object 
 2   cuisine          500 non-null    object 
 3   rating           500 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.8+ KB


In [16]:
print("\n[ S4 ] Combining All Tables\n")

orders_users_merge = pd.merge(
    orders_data,
    users_data,
    on="user_id",
    how="left"
)

complete_dataset = pd.merge(
    orders_users_merge,
    restaurant_data,
    on="restaurant_id",
    how="left"
)

print(complete_dataset.head())
print("\nMerged Dataset Overview:")
complete_dataset.info()


[ S4 ] Combining All Tables

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3 

In [17]:
print("\n[ S5 ] Cleaning Columns & Exporting\n")

if "restaurant_name_x" in complete_dataset.columns:
    complete_dataset.drop("restaurant_name_x", axis=1, inplace=True)

complete_dataset.rename(
    columns={"restaurant_name_y": "restaurant_name"},
    inplace=True
)

complete_dataset["order_date"] = pd.to_datetime(
    complete_dataset["order_date"],
    dayfirst=True
)

complete_dataset.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print("✅ Dataset created: final_food_delivery_dataset.csv")


[ S5 ] Cleaning Columns & Exporting

✅ Dataset created: final_food_delivery_dataset.csv
